# Create Composite 10m DEM

**Timm Nawrocki**  
Alaska Center for Conservation Science  
2019-03-24

In [1]:
# -*- coding: utf-8 -*-
# ---------------------------------------------------------------------------
# Create Composite 10m DEM
# Author: Timm Nawrocki
# Created on: 2019-03-24
# Usage: Must be executed as a Jupyter Notebook in an ArcGIS Pro Python 3 installation.
# Description: "Create Composite 10m DEM" generates a Digital Elevation Model (DEM) for the Hydrographic Area of Influence of the Bristol Bay Lowlands, Ahklun Mountains, and Game Management Unit 17 by combining the 10m Arctic DEM with a downscaled 60m USGS 3D Elevation Program DEM.
# ---------------------------------------------------------------------------

In [2]:
# Import packages
import arcpy
from arcpy.sa import *
import os

# Set overwrite option
arcpy.env.overwriteOutput = True

# Set root directory
drive = 'E:/'
root_directory = os.path.join(drive, 'VegetationEcology/BristolBay_Vegetation/Project_GIS')

# Set arcpy working environment
arcpy.env.workspace = os.path.join(root_directory, 'BristolBay_Vegetation.gdb')

# Define script paths
script_directory = os.path.join(drive, 'Repositories/southwest-alaska-moose/modules')
arcpy_geoprocessing_script = os.path.join(script_directory, 'arcpy_geoprocessing.py')

# Define input datasets
arcticDEM_tile_folder = os.path.join(root_directory, 'Data_Input/source_data/elevation/arctic_10m_DEM/raster_tiles')
usgs_60m_dem = os.path.join(root_directory, 'Data_Input/source_data/elevation/USGS3DEP_60m_DEM/Alaska_USGS3DEP_Elevation_60m_AKALB_20181106.tif')
hydrographic_area = os.path.join(arcpy.env.workspace, 'SouthwestAlaska_HydrographicArea')
alaska_boundary = os.path.join(root_directory, 'Data_Input/source_data/boundaries/Alaska_63360_ExcludingSmallIslands.shp')

# Define output raster
arctic_dem_composite = os.path.join(root_directory, 'Data_Input/source_data/elevation/arctic_10m_DEM/raster_composite/arctic_10m_DEM_composite.tif')
bristol_bay_dem = os.path.join(root_directory, 'Data_Input/predictor_env/bristolBay_10m_DEM/bristolBay_10m_DEM.tif')

In [ ]:
# Import and execute arcpy_geoprocessing.py
try:
    exec(open(arcpy_geoprocessing_script).read())
except:
    print("Error loading arcpy_geoprocessing.py; ensure that script directory is correct:")
    print(script_directory)
    quit()

In [ ]:
# Define function to create composite DEM
def create_composite_dem(**kwargs):
    """
    Description: creates a DEM from Arctic DEM 10m tiles and the USGS 3DEP 60m DEM
    Inputs: 'tile_folder' -- a folder containing the raster tiles for the 10m Arctic DEM
            'input_array' -- an array containing game management units, unified ecoregions, and watersheds (10 digit hydrologic units)
            'output_array' -- the output should be an array with a full feature class and a clipped feature class
    """
    # Parse key word argument inputs
    tile_folder = kwargs['tile_folder']
    usgs_60m_dem = kwargs['input_array'][0]
    hydrographic_area = kwargs['input_array'][1]
    alaska_boundary = kwargs['input_array'][2]
    arctic_dem_composite = kwargs['output_array'][0]
    bristol_bay_dem = kwargs['output_array'][1]
    # Define intermediate datasets
    mosaic_location, mosaic_name = os.path.split(arctic_dem_composite)
    mask_feature = 'mask_feature'
    arctic_dem_composite_akalb = os.path.join(mosaic_location, 'arctic_10m_DEM_composite_AKALB.tif')
    usgs_60m_dem_resample = os.path.join(os.path.split(usgs_60m_dem)[0], 'usgs_60m_dem_resample.tif')
    # Create a list of Arctic DEM raster tiles
    tile_list = []
    for file in os.listdir(tile_folder):
        if file.endswith('tif'):
            tile_list.append(os.path.join(tile_folder, file))
    print('Process will form composite from {0} raster tiles...'.format(len(tile_list)))
    # Define projection for all raster tiles as National Snow and Ice Data Center Sea Ice Polar Stereographic North (WGS84)
    polar_stereographic = arcpy.SpatialReference(3413) #WGS 1984 NSIDC Sea Ice Polar Stereographic North
    for raster in tile_list:
        arcpy.DefineProjection_management(raster, polar_stereographic)
    print('Defined projection for {0} raster tiles...'.format(len(tile_list)))
    # Mosaic raster tiles to new raster
    arcpy.MosaicToNewRaster_management(tile_list,
                                       mosaic_location,
                                       mosaic_name,
                                       polar_stereographic,
                                       '32_BIT_FLOAT',
                                       '10',
                                       '1',
                                       'MAXIMUM',
                                       'FIRST')
    print('Mosaicked raster tiles to new raster...')
    # Set snap raster
    arcpy.env.snapRaster = usgs_60m_dem
    # Project Arctic DEM composite to NAD 1983 Alaska Albers
    akalb = arcpy.SpatialReference(3338) #NAD 1983 Alaska Albers
    transform_method = 'WGS_1984_(ITRF00)_To_NAD_1983'
    arcpy.ProjectRaster_management(arctic_dem_composite,
                                   arctic_dem_composite_akalb,
                                   akalb, 'BILINEAR',
                                   '10',
                                   transform_method)
    print('Projected Arctic DEM composite to NAD 1983 Alaska Albers...')
    # Clip hydrographic area to Alaska boundary to form mask
    arcpy.Clip_analysis(hydrographic_area, alaska_boundary, mask_feature)
    print('Created mask feature class...')
    # Downscale USGS 3DEP 60m DEM to 10m
    usgs_60m_extract = ExtractByMask(usgs_60m_dem, mask_feature)
    arcpy.Resample_management(usgs_60m_extract, usgs_60m_dem_resample, '10 10', 'BILINEAR')
    print('Downsampled USGS 3DEP 60m DEM to 10m...')
    # Fill holes in the Arctic DEM with the resampled USGS DEM
    filled_raster = Con(IsNull(arctic_dem_composite_akalb), usgs_60m_dem_resample, arctic_dem_composite_akalb)
    print('Filled NoData cells in Arctic DEM with downsampled USGS DEM...')
    extract_raster = ExtractByMask(filled_raster, mask_feature)
    print('Extracted DEM to mask feature class...')
    integer_raster = Int(extract_raster + 0.5)
    print('Converted floating values to integers...')
    arcpy.CopyRaster_management(integer_raster, bristol_bay_dem, '', '', '-32768', 'NONE', 'NONE', '16_BIT_SIGNED', 'NONE', 'NONE', 'TIFF', 'NONE')
    print('Created output Bristol Bay DEM...')
    # Delete intermediate dataset
    arcpy.Delete_management(mask_feature)
    out_process = arcpy.Delete_management(usgs_60m_dem_resample)
    return out_process

In [ ]:
# Define input and output arrays
create_dem_inputs = [usgs_60m_dem, hydrographic_area, alaska_boundary]
create_dem_outputs = [arctic_dem_composite, bristol_bay_dem]

# Create key word arguments
create_dem_kwargs = {'tile_folder' : arcticDEM_tile_folder,
                     'input_array' : create_dem_inputs,
                     'output_array' : create_dem_outputs
                    }

# Process the create polygon function with the point array
arcpy_geoprocessing(create_composite_dem, **create_dem_kwargs)